<a href="https://colab.research.google.com/github/barbosacesar/Intelig-ncia-Computacional/blob/main/projeto_disciplina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --no-cache-dir numpy==1.26.0 pandas ipywidgets opencv-python-headless fer matplotlib moviepy

In [ ]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=9a36c07f8e6dde4f48548ba95e45a4c794e638a1f93209307fe59a3cb2277729
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
# Montar Google Drive para acessar seus arquivos
from google.colab import drive
drive.mount('/content/drive')

# Mudar diretório para sua pasta de trabalho
%cd "/content/drive/MyDrive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript
import time
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import uuid
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import zipfile
import os
from pathlib import Path

# Importando DeepFace
from deepface import DeepFace

EMOCOES_INFO = {
    'happy': ('Feliz', '😄', '#4CAF50'),
    'sad': ('Triste', '😢', '#1E88E5'),
    'angry': ('Bravo', '😠', '#E53935'),
    'surprise': ('Surpreso', '😲', '#FFEB3B'),
    'neutral': ('Neutro', '😐', '#9E9E9E'),
    'fear': ('Medo', '😨', '#8E24AA'),
    'disgust': ('Nojo', '🤢', '#6D4C41'),
    'neutral': ('Neutro', '😐', '#9E9E9E')
}

def emocao_info(emocao):
    return EMOCOES_INFO.get(str(emocao).lower(), ('Desconhecida', '❓', '#000000'))

saida = widgets.Output()

def get_facial_attributes(filename, conf_threshold=0.3):
    try:
        result = DeepFace.analyze(img_path=filename, actions=['emotion'], enforce_detection=False)
        if isinstance(result, list):
            result = result[0]  # se for lista, pega primeiro resultado
        emotions = result["emotion"]
        dominant_emotion = result["dominant_emotion"]
        confidence = emotions[dominant_emotion] / 100

        if confidence < conf_threshold:
            # Confiança baixa, ignorar resultado
            return []

        return [{
            "facial_expression": dominant_emotion,
            "confidence": confidence,
            "emotions": emotions
        }]
    except Exception as e:
        print("Erro na análise facial:", e)
        return []


def mostrar_resultado(result, tempo_inferencia=None):
    with saida:
        clear_output(wait=True)
        try:
            df = pd.DataFrame(result)
            if df.empty:
                print("Nenhuma face detectada.")
                return

            df = df.sort_values(by='confidence', ascending=False).reset_index(drop=True)
            df['Emoção (PT)'] = df['facial_expression'].apply(lambda e: emocao_info(e)[0])
            df['Emoji'] = df['facial_expression'].apply(lambda e: emocao_info(e)[1])
            df['Cor'] = df['facial_expression'].apply(lambda e: emocao_info(e)[2])

            tabela_html = """
            <style>
            .barra-conf {
                height: 18px;
                text-align: right;
                padding-right: 5px;
                color: white;
                font-weight: bold;
                border-radius: 4px;
            }
            table {
                border-collapse: collapse;
                width: 100%;
                font-family: Arial, sans-serif;
            }
            th, td {
                border: 1px solid #ddd;
                padding: 8px;
                text-align: center;
            }
            th {
                background-color: #00008B;
                color: white;
            }
            </style>
            <table>
            <thead>
                <tr>
                    <th>Emoção</th>
                    <th>Emoji</th>
                    <th>Confiança</th>
                </tr>
            </thead>
            <tbody>
            """

            for _, row in df.iterrows():
                perc = int(row['confidence'] * 100)
                cor = row['Cor']
                tabela_html += f"""
                <tr>
                    <td style='color:{cor}; font-weight:bold'>{row['Emoção (PT)']}</td>
                    <td style='font-size:24px;'>{row['Emoji']}</td>
                    <td>
                        <div class='barra-conf' style='width:{perc}%; background-color:{cor};'>{perc}%</div>
                    </td>
                </tr>
                """

            tabela_html += "</tbody></table>"
            display(HTML(tabela_html))

            emocao_predominante = df.loc[0, 'Emoção (PT)']
            emoji_predominante = df.loc[0, 'Emoji']
            conf_predominante = int(df.loc[0, 'confidence'] * 100)
            cor_predominante = df.loc[0, 'Cor']

            display(HTML(f"""
                <h3 style='text-align:center;'>
                    Emoção predominante: <span style='color:{cor_predominante};'>{emocao_predominante}</span> {emoji_predominante}
                    <br><small>Confiança: {conf_predominante}%</small>
                </h3>
            """))

            if tempo_inferencia is not None:
                display(HTML(f"<p><b>Tempo de inferência:</b> {tempo_inferencia:.3f} segundos</p>"))

        except Exception as e:
            print("Erro ao mostrar resultado:", e)
def on_upload_change_zip(change):
    if not uploader.value:
        return

    arquivo = next(iter(uploader.value.values()))
    nome_zip = arquivo['metadata']['name']
    conteudo = arquivo['content']

    # Salvar zip temporariamente
    with open(nome_zip, 'wb') as f:
        f.write(conteudo)

    # Descompactar em pasta temporária
    pasta_temp = f"/tmp/extracted_{uuid.uuid4().hex}"
    os.makedirs(pasta_temp, exist_ok=True)

    with zipfile.ZipFile(nome_zip, 'r') as zip_ref:
        zip_ref.extractall(pasta_temp)

    print(f"Arquivos extraídos em: {pasta_temp}")

    # Agora analisar todas as imagens dentro dessa pasta
    analisar_pasta_emocoes(pasta_temp)

    # Limpar upload para nova seleção
    uploader.value.clear()

def analisar_pasta_emocoes(caminho_pasta):
    pasta = Path(caminho_pasta)
    imagens = list(pasta.rglob("*.jpg")) + list(pasta.rglob("*.jpeg")) + list(pasta.rglob("*.png"))

    if not imagens:
        with saida:
            clear_output(wait=True)
            print("Nenhuma imagem encontrada na pasta.")
        return

    contagem_emocoes = Counter()
    total_start = time.time()

    with saida:
        clear_output(wait=True)
        print(f"Analisando {len(imagens)} imagens...")

    for img_path in imagens:
        try:
            resultado = get_facial_attributes(str(img_path))
            if resultado:
                emocao = resultado[0]["facial_expression"]
                contagem_emocoes[emocao] += 1
        except Exception as e:
            print(f"Erro ao analisar {img_path.name}: {e}")

    total_end = time.time()

    with saida:
        clear_output(wait=True)
        total = sum(contagem_emocoes.values())
        if total == 0:
            print("Nenhuma emoção detectada nas imagens.")
            return

        print(f"Total de imagens analisadas: {len(imagens)}")
        print(f"Total de emoções detectadas: {total}")
        print(f"Tempo total: {total_end - total_start:.2f} segundos\n")

        print("Contagem por emoção:")
        for emocao, count in contagem_emocoes.items():
            pt, emoji, _ = emocao_info(emocao)
            print(f"{emoji} {pt:<10}: {count} ({count/total:.1%})")

        # Exibir gráfico
        try:
            labels = []
            values = []
            cores = []

            for emocao, count in contagem_emocoes.items():
                pt, emoji, cor = emocao_info(emocao)
                labels.append(f"{emoji} {pt}")
                values.append(count)
                cores.append(cor)

            plt.figure(figsize=(10, 5))
            bars = plt.bar(labels, values, color=cores)
            plt.title("Distribuição das Emoções Detectadas")
            plt.xlabel("Emoções")
            plt.ylabel("Frequência")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print("Erro ao gerar gráfico:", e)
def on_upload_change(change):
    if not uploader.value:
        return

    arquivos = list(uploader.value.values())  # sem limite
    contagem_emocoes = Counter()
    total_start = time.time()

    with saida:
        clear_output(wait=True)
        print(f"{len(arquivos)} arquivo(s) enviado(s). Iniciando análise...")

    for arquivo in arquivos:
        filename = arquivo['metadata']['name']
        content = arquivo['content']
        filepath = f"/tmp/{filename}"

        with open(filepath, 'wb') as f:
            f.write(content)

        # Se for ZIP, extrair e analisar imagens dentro
        if filename.lower().endswith('.zip'):
            pasta_temp = f"/tmp/extracted_{uuid.uuid4().hex}"
            os.makedirs(pasta_temp, exist_ok=True)
            try:
                with zipfile.ZipFile(filepath, 'r') as zip_ref:
                    zip_ref.extractall(pasta_temp)
            except Exception as e:
                print(f"Erro ao descompactar {filename}: {e}")
                continue

            imagens = list(Path(pasta_temp).rglob("*.jpg")) + list(Path(pasta_temp).rglob("*.jpeg")) + list(Path(pasta_temp).rglob("*.png"))
            if not imagens:
                print(f"Nenhuma imagem encontrada dentro do ZIP {filename}")
                continue

            for img_path in imagens:
                try:
                    resultado = get_facial_attributes(str(img_path))
                    if resultado:
                        emocao = resultado[0]["facial_expression"]
                        contagem_emocoes[emocao] += 1
                except Exception as e:
                    print(f"Erro ao analisar {img_path.name}: {e}")

        # Se for imagem, analisar diretamente
        elif filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                resultado = get_facial_attributes(filepath)
                if resultado:
                    emocao = resultado[0]["facial_expression"]
                    contagem_emocoes[emocao] += 1
            except Exception as e:
                print(f"Erro ao analisar {filename}: {e}")

        else:
            print(f"{filename} ignorado (formato inválido).")

    total_end = time.time()

    with saida:
        clear_output(wait=True)
        total = sum(contagem_emocoes.values())
        if total == 0:
            print("Nenhuma emoção detectada nas imagens.")
            return

        print(f"Total de emoções detectadas: {total}")
        print(f"Tempo total: {total_end - total_start:.2f} segundos\n")

        print("Contagem por emoção:")
        for emocao, count in contagem_emocoes.items():
            pt, emoji, _ = emocao_info(emocao)
            print(f"{emoji} {pt:<10}: {count} ({count/total:.1%})")

        # Exibir gráfico
        try:
            labels = []
            values = []
            cores = []

            for emocao, count in contagem_emocoes.items():
                pt, emoji, cor = emocao_info(emocao)
                labels.append(f"{emoji} {pt}")
                values.append(count)
                cores.append(cor)

            plt.figure(figsize=(10, 5))
            bars = plt.bar(labels, values, color=cores)
            plt.title("Distribuição das Emoções Detectadas")
            plt.xlabel("Emoções")
            plt.ylabel("Frequência")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print("Erro ao gerar gráfico:", e)

    uploader.value.clear()

def capture_image():
    display(Javascript('''
    async function capture() {
        const oldDiv = document.getElementById('camera-capture-div');
        if (oldDiv) {
            oldDiv.remove();
        }

        const div = document.createElement('div');
        div.id = 'camera-capture-div';

        const video = document.createElement('video');
        const canvas = document.createElement('canvas');
        const context = canvas.getContext('2d');
        const button = document.createElement('button');
        button.textContent = 'Capture';

        document.body.appendChild(div);
        div.appendChild(video);
        div.appendChild(button);

        const stream = await navigator.mediaDevices.getUserMedia({ video: true });
        video.srcObject = stream;
        await video.play();

        await new Promise((resolve) => button.onclick = resolve);

        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        context.drawImage(video, 0, 0, canvas.width, canvas.height);

        stream.getTracks().forEach(track => track.stop());
        div.remove();

        return canvas.toDataURL('image/jpeg');
    }
    capture();
    '''))

    data = eval_js("capture()")
    if not data.startswith("data:image/jpeg;base64,"):
        raise ValueError("Erro ao capturar imagem da webcam")
    encoded = data.split(',')[1]
    img_data = b64decode(encoded)
    filename = f"captura_{uuid.uuid4().hex}.jpg"
    with open(filename, 'wb') as f:
        f.write(img_data)
    return filename

def capturar_webcam(b):
    with saida:
        clear_output(wait=True)
        print("Abrindo webcam... Clique no botão 'Capture' para tirar a foto.")
        try:
            filename = capture_image()
            print("Imagem capturada. Iniciando análise...")

            start = time.time()
            result = get_facial_attributes(filename)
            end = time.time()

            mostrar_resultado(result, end - start)
        except Exception as e:
            print("Erro durante a captura/análise:", e)

def mostrar_menu():
    global uploader
    uploader = widgets.FileUpload(
        accept='.png,.jpg,.jpeg,.zip,.rar',
        multiple=False,
        description='Enviar Imagem',
        style={'button_color': '#2196F3', 'font_weight': 'bold', 'font_size': '16px'}
    )
    uploader.observe(on_upload_change, names='value')

    botao_webcam = widgets.Button(
        description="Capturar Webcam",
        button_style='info',
        layout=widgets.Layout(width='160px'),
        style={'font_weight': 'bold', 'font_size': '16px'}
    )
    botao_webcam.on_click(capturar_webcam)

    botoes = widgets.HBox([uploader, botao_webcam],
                         layout=widgets.Layout(justify_content='center', gap='20px', padding='10px'))

    display(HTML("<h2 style='text-align:center;'>Análise de Emoções Faciais</h2>"))
    display(HTML("<p style='text-align:center; font-size:16px;'>Envie uma imagem ou capture da webcam</p>"))
    display(botoes)
    display(saida)

mostrar_menu()

Output()